# メタデータを検証する

研究データに付与されているメタデータを検証し、その結果をレポートします。  
検証を行うことで、適切なメタデータが研究データに付与されているかを確認することができます。  

検証結果は必ず同期する必要はありません。検証結果を残さずに確認したい場合は、同期せず破棄してください。この環境で検証結果を確認いただいたあとに、GIN-forkへ同期するか破棄するか選択いただけます。  
検証結果を同期する場合は、研究リポジトリの直下の**validation_results**フォルダに格納されます。タスク実行後に検証結果を確認する際はvalidation_resultsフォルダ配下の**results.json**をご確認ください。  
同期しない場合は、検証結果は破棄されます。

validation_results配下のファイルのサンプルや確認方法について、詳しくは[マニュアル](https://support.rdm.nii.ac.jp/)を参照ください。  

validation_resultsフォルダの構成は以下です。  
![validation_results_dir](../../images/validation_results_dir.png)

以下の点にご留意ください。  
- 実験実行環境の初期セットアップを終えていない場合、エラーが発生し、検証を行えません。メタデータの検証を行う前に、研究フロートップページに戻り、『実験実行環境を作成する』から実験実行環境を作成し、構築した環境で「初期セットアップを行う」を実行してください。
- ユーザー情報の個人URLにORCIDを入力していない場合、検証NGになります。

研究フロートップページに戻る場合は[こちら](../../base_FLOW.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、当実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 研究リポジトリのメタデータを用意する

メタデータを検証するための準備として、研究リポジトリのメタデータの取得と整形を行います。

In [ ]:
import os
from urllib import parse
import requests
import json
from dg_packager.ro_generator.gin_ro_generator import GinRoGenerator
from dg_packager.error.error import JsonValidationError, RoPkgError
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util

try:
    # get current branch name
    os.chdir(os.environ['HOME'])
    result = !git branch --contains
    current_branch = result[0].replace('* ', '')

    # get repository id
    from WORKFLOWS.utils.repository_id import repository_id
    repo_id = repository_id.get_repo_id()

    # get token for ginfork
    from WORKFLOWS.utils.token import token
    ginfork_token = token.get_ginfork_token()

    # generate url_for_request_metadata
    path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
    os.chdir(path_flow_root)
    from util.scripts import utils
    params = {}
    with open(utils.fetch_param_file_path(), mode='r') as f:
        params = json.load(f)
    parse_result = parse.urlparse(params['siblings']['ginHttp'])
    url_for_request_metadata = parse.urlunparse((
        parse_result.scheme, 
        parse_result.netloc, 
        'api/v1/repos/' + repo_id + '/' + current_branch + '/metadata',
        '',
        'token=' + ginfork_token,
        ''
    ))

    # get raw metadata from ginfork & package
    response = requests.get(url_for_request_metadata)
    result = response.json()
except Exception as e:
    display_util.display_err('GIN-forkへの研究リポジトリのメタデータの問い合わせに失敗しました。担当者にお問い合わせください。')
    display_util.display_err(str(e))
else:
    if response.status_code == requests.codes.ok:
        raw_metadata = result
        # packaging
        try:
            ro_crate = GinRoGenerator.Generate(raw_metadata)
            gen_rocrate = True
            display_util.display_info('正常にメタデータを取得できました。次の処理にお進みください。')
        except JsonValidationError as e:
            # If given Raw Metadata to Function is invalid format, exception occurs.(derived dg-packager)
            display_util.display_err('メタデータにの形式に不備があります。ご確認ください。')
            display_util.display_err(str(e))
        except RoPkgError as e:
            # If each value of metadata is invalid on checking property, exception occurs.(derived SDK Library)
            if "{'<ginfork.GinMonitoring #ginmonitoring>': {'experimentPackageList': 'This property is required, but not found.'}" in str(e):
                msg = "実験が開始されていない可能性があります。<br>検証を行う前に、研究フロートップページに戻り、"\
                "『実験実行環境を作成する』から実験実行環境を作成し、構築した環境で「初期セットアップを行う」を実行してください。"
                display_util.display_err(msg)
            else:
                display_util.display_err('メタデータに不備があります。ご確認ください。')
                display_util.display_err(str(e))
        except Exception as e:
            display_util.display_err('メタデータに不備があります。ご確認ください。')
            display_util.display_err(str(e))
    elif response.ok == False:
        display_util.display_err('異常が発生しました。担当者にお問い合わせください。')
        display_util.display_err(result['message'])

## 2. メタデータを検証する

メタデータの検証と検証結果の生成を行う検証サービスに、メタデータの検証を依頼します。

In [ ]:
from urllib import parse
import requests
import os
import json

if 'gen_rocrate' not in locals():
    display_util.display_err("手順1が実行されていないか、もしくは正常終了していないため実行できません。")
else:
    try:
        # generate url_for_request_validation
        path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
        os.chdir(path_flow_root)
        from util.scripts import utils
        params = {}
        with open(utils.fetch_param_file_path(), mode='r') as f:
            params = json.load(f)
        url_for_request_validation = parse.urlunparse((
            params['dgCore']['Scheme'],
            params['dgCore']['Netloc'], 
            'validate',
            '',
            '',
            ''
        ))
        # request validation
        headers = {'content-type': 'application/json'}
        response = requests.post(url_for_request_validation, data=json.dumps(ro_crate), headers=headers)
        result = response.json()
    except Exception as e:
        display_util.display_err('検証サービスに接続できません。')
        display_util.display_err(str(e))
    else:
        if response.status_code == requests.codes.ok:
            os.chdir(os.environ['HOME'])
            from WORKFLOWS.utils.tmp_validation import tmp_validation
            request_id = result['request_id']
            tmp_validation.save_request_id(request_id)
            validate_requested = True
            display_util.display_info('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
        elif response.ok == False:
            display_util.display_err('異常が発生しました。担当者にお問い合わせください。')
            display_util.display_err(result['message'])

## 3. 検証結果を確認する

検証サービスから検証結果を取得し、結果を出力します。

In [ ]:
import os
import json
import requests
from urllib import parse
import time
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from WORKFLOWS.utils.tmp_validation import tmp_validation

def on_click_callback(clicked_button: widgets.Button) -> None:
    clear_output()
    display_util.display_info('検証結果の出力を削除しました。\n再度確認したい場合は、次のセルを実行する前にこのセルを再実行してください。')

def show_results():
    display_util.display_warm('検証の結果、メタデータに以下の不備が見つかりました。確認後、「確認を完了する」ボタンをクリックして次にお進みください。'\
                  '\n「確認を完了する」ボタンがクリックされていない場合は、検証結果を含んだこのノートブックがリポジトリに同期されます。')
    button = widgets.Button(description='確認を完了する')
    button.on_click(on_click_callback)
    !cat $result_file_path
    display(button)

if 'validate_requested' not in locals():
    display_util.display_err("手順2が実行されていないか、もしくは正常終了していないため実行できません。")
else:    
    request_id = tmp_validation.get_request_id()
    counter = 5
    while counter >0:
        try:
            # generate url_for_get_validation_results
            path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
            os.chdir(path_flow_root)
            from util.scripts import utils
            params = {}
            with open(utils.fetch_param_file_path(), mode='r') as f:
                params = json.load(f)
            url_for_get_validation_results = parse.urlunparse((
                params['dgCore']['Scheme'],
                params['dgCore']['Netloc'], 
                request_id,
                '',
                '',
                ''
            ))
            # request get validation results
            response = requests.get(url_for_get_validation_results)
            result = response.json()
            counter -= 1
            clear_output()
        except Exception as e:
            display_util.display_err('検証サービスに接続できません。')
            display_util.display_err(str(e))
            break
        else:
            if response.status_code == requests.codes.ok:
                status = result['status']
                if status == 'UNKNOWN':
                    display_util.display_err('リクエストID：' + request_id + 'の状況が読み込めませんでした。')
                    break
                elif any([status == 'QUEUED', status == 'RUNNING']):
                    display_util.display_warm('リクエストID：' + request_id + 'は検証完了していません。時間をおいて再度確認します。')
                    time.sleep(10)
                    continue
                elif status == 'COMPLETE':
                    tmp_validation.save_verification_results(result)
                    get_results = True
                    display_util.display_info('メタデータ（人物、所属組織、ファイル、データセット、リポジトリ情報）、研究データの容量、実験パッケージの構成状態は適切に管理されています。次にお進みください。')
                    break
                elif status == 'FAILED':
                    tmp_validation.save_verification_results(result)
                    result_file_path = tmp_validation.fetch_validation_results_file_path()
                    get_results = True
                    show_results()
                    break
                elif status == 'EXECUTOR_ERROR':
                    display_util.display_err('正常に検証を実行できませんでした。')
                    break
                elif status == 'CANCELING':
                    display_util.display_err('リクエストID：' + request_id + 'は現在キャンセル中です。')
                    break
                elif status == 'CANCELED':
                    display_util.display_err('リクエストID：' + request_id + 'はキャンセルされました。')
                    break

            elif response.ok == False:
                display_util.display_err('異常が発生しました。担当者にお問い合わせください。')
                display_util.display_err(result['message'])
                        
    else:
        clear_output()
        display_util.display_warm('検証に時間がかかっています。時間をおいて再度このセルを実行してください。')

## 4. 研究リポジトリに同期する

このタスクの実行結果を研究リポジトリに同期します。  
検証結果を同期するか破棄するかは、「4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する」で選択できます。

### 4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する

In [ ]:
import panel as pn

pn.extension()
column = pn.Column()

def save_selection_result(event):
    global need_sync
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    need_sync = True if select.value == 1 else False

if 'get_results' not in locals():
    display_util.display_err("手順3が実行されていないか、もしくは正常終了していないため実行できません。")
else:
    select = pn.widgets.Select(name='検証結果を同期するか破棄するかを選択した後、完了ボタンをクリックしてください。', options={'同期する': 1, '同期せずに破棄する':2})
    done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
    done_button.on_click(save_selection_result)
    column.append(select)
    column.append(done_button)
column

### 4.2. 研究リポジトリに同期する

研究リポジトリにこのタスクの実行結果を同期します。  
「4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する」で同期せずに破棄するを選択した場合は、検証結果は同期されずこのファイルの実行結果のみが同期されます。

In [ ]:
import os
from IPython.display import display, Javascript
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])
from WORKFLOWS.utils.tmp_validation import tmp_validation

if 'need_sync' not in locals():
    display_util.display_err("手順4-1が実行されていないか、もしくは正常終了していないため実行できません。")
else:
    tmp_validation.operate_validation_results(need_sync)
    git_path = ['/home/jovyan/WORKFLOWS/FLOW/02_experimental_phase/base_validate_metadata.ipynb']
    if need_sync == True:
        validation_result_path = tmp_validation.fetch_validation_result_path()
        git_path.append(validation_result_path)

    display(Javascript('IPython.notebook.save_checkpoint();'))
    is_ok = utils.syncs_with_repo(git_path=git_path, gitannex_path=[], gitannex_files=[], message='メタデータ検証', get_paths=[])
    if is_ok:
        tmp_validation.delete_verification_results_and_request_id()

tickets = ['gen_rocrate', 'validate_requested', 'get_results', 'need_sync']
for ticket in tickets:
    if ticket in locals():
        del locals()[ticket]

## 5. 研究フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究フロートップページに戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))